In [1]:
!pip install transformers peft torch datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!huggingface-cli login
# from huggingface_hub import notebook_login
# notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `mytoken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mytoken`


In [3]:
# 3 hours spent trying to figure out how to fine-tune with insufficient vram.
# the solution? just pay for a better colab GPU, apparently

In [3]:
import json
from datasets import Dataset

# Load JSON data
def read_in_data(json_file):
    with open(json_file, "r") as f:
        return json.load(f)

# Read and format dataset
JSON_FILE = "prompts_AHandfulOfDust_n1.json"
jdata = read_in_data(JSON_FILE)

dataset = Dataset.from_dict({"text": [d['prompt'] + d['ground_truth'] for d in jdata[5:35]]})

# Split dataset for training & evaluation
dataset = dataset.train_test_split(test_size=0.2)


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
print(model)
# 32 mistralattn layers, mlp

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [6]:
for param in model.parameters():
    param.requires_grad = False

for layer in model.model.layers[-2:]:
    for param in layer.parameters():
        param.requires_grad = True


trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 436224000


In [7]:
tokenizer.pad_token = tokenizer.unk_token

def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Labels should be the same as input_ids
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [10]:
def generate_response(prompt):
  model.eval()
  with torch.no_grad():
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)
generate_response("Hello!")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Hello! I'm a new member of the community and I'm excited to be here. I'm a writer and I love to read and write about science fiction and fantasy. I'm also a huge fan of video games and I love to play them. I'm always looking for new games to play and new stories to read. If you have any recommendations, please let me know!"

In [8]:
for i, (b, a) in enumerate(zip(before_weights, after_weights)):
    if not torch.allclose(b, a):
        print(f"Parameter {i} changed!")
    else:
      print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [11]:
from accelerate import Accelerator
from torch.utils.data import DataLoader
from torch.optim import AdamW
import os

lr = 1e-5
epochs = 3
batch_size = 4


accelerator = Accelerator(mixed_precision="no")
train_loader = DataLoader(tokenized_datasets['train'], batch_size=batch_size, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# thank you, adamw, for randomly deciding to replace entire matrices of weights
# with nan. thank you for not raising any sort of error when you did that, too.

model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)
loss_history = []
model.train()
for epoch in range(epochs):
    for batch in train_loader:
        outputs = model(**batch)
        loss = outputs.loss

        if not torch.isfinite(loss):
          raise Exception("wee woo wee woo infinite loss alert")
        loss_history.append(loss.item())
        layer = model.model.layers[-1]

        accelerator.backward(loss)

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()
        # raise Exception("wawa")
print("loss", loss_history)

loss [2.631479024887085, 2.6315736770629883, 2.78857421875, 2.7513482570648193, 2.631514549255371, 2.63163161277771, 2.6316049098968506, 2.631542444229126, 2.631472587585449, 2.631558418273926, 2.751286268234253, 2.788635730743408, 2.788564682006836, 2.6315183639526367, 2.6315231323242188, 2.6315298080444336, 2.7512850761413574, 2.631463050842285]


In [12]:
for j in jdata[:5]:
  prompt = j['prompt']
  print("===========================================")
  print(prompt)
  print(generate_response(prompt))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dialogue History: John Beaver: Was anyone hurt?
Instruction: Continue the dialogue for 1 lines.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dialogue History: John Beaver: Was anyone hurt?
Instruction: Continue the dialogue for 1 lines.

John Beaver: No, but I'm sure they'll be fine.

John Beaver: I'm sure they'll be fine.

John Beaver: I'm sure they'll be fine.

John Beaver: I'm sure they'll be fine.

John Beaver: I'm sure they'll be fine.

John Beaver: I'm sure they'll be fine.

Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dia

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dialogue History: John Beaver: Was anyone hurt?
Mrs. Beaver: No one, I am thankful to say, except two housemaids who lost their heads and jumped through a glass roof into the paved court. They were in no danger. The fire never reached the bedrooms, I am afraid. Still, they are bound to need doing up, everything black with smoke and drenched in water and luckily they had that old-fashioned sort of extinguisher that ruins _everything_. One really cannot complain. The chief rooms were _completely_ gutted and everything was insured. Sylvia Newport knows the people. I must get on to them this morning before that ghoul Mrs Shutter snaps them up.
Instruction: Continue the 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.

Dialogue History: John Beaver: Was anyone hurt?
Mrs. Beaver: No one, I am thankful to say, except two housemaids who lost their heads and jumped through a glass roof into the paved court. They were in no danger. The fire never reached the bedrooms, I am afraid. Still, they are bound to need doing up, everything black with smoke and drenched in water and luckily they had that old-fashioned sort of extinguisher that ruins _everything_. One really cannot complain. The chief rooms were _completely_ gutted and everything was insured. Sylvia Newport knows the people. I must get on to them this morning before that ghoul Mrs Shutter snaps them up.
Mrs. Beaver: Heavens, how 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Character Info: Nanny: A nurturing and steady figure in Tony Last's household, providing a stable presence in the tumultuous events of the family.
Tony Last: The novel's protagonist, a gentleman devoted to his estate, Hetton Abbey. His naive attachment to tradition and his wife, Brenda, leads to his ultimate downfall.
Brenda: Tony Last's wife, who becomes dissatisfied with her marriage and has an affair with John Beaver, ultimately prioritizing her own social standing over her marriage.
Ben: A worker in the Last household, representing the invisible labor that sustains upper-class families.
John Andrew: Tony and Brenda Last's young son, whose tragic death marks the turning point of the novel and accelerates Tony's misfortune.

Dialogue History: Tony Last: Has her ladyship been called yet?
Instruction: Continue the dialogue for 1 lines.
Nanny: Yes, sir. She's on her way.
Character Info: Nanny: A nurturing and steady figure in Tony Last's household, providing a stable presence in the tum

In [13]:
# Save
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# Load
# from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(model_name)
# fine_tuned_model = PeftModel.from_pretrained(base_model, "./fine_tuned_model")


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/tokenizer.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
!zip -r fine_tuned_model.zip ./fine_tuned_model

  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/tokenizer_config.json (deflated 68%)
  adding: fine_tuned_model/model-00001-of-00003.safetensors